

#Heuristique for flow shop problem



In [ ]:
import numpy as np


In [ ]:
def evaluate_sequence(sequence, processing_times):
    _, num_machines = processing_times.shape
    num_jobs = len(sequence)

    # Check if the sequence is empty
    if num_jobs == 0:
        # Return a default value (you may choose 0 or another suitable value)
        return 0

    completion_times = np.zeros((num_jobs, num_machines))

    # Calculate the completion times for the first machine
    completion_times[0][0] = processing_times[sequence[0]][0]
    for i in range(1, num_jobs):
        completion_times[i][0] = completion_times[i-1][0] + processing_times[sequence[i]][0]

    # Calculate the completion times for the remaining machines
    for j in range(1, num_machines):
        completion_times[0][j] = completion_times[0][j-1] + processing_times[sequence[0]][j]
        for i in range(1, num_jobs):
            completion_times[i][j] = max(completion_times[i-1][j], completion_times[i][j-1]) + processing_times[sequence[i]][j]

    # Return the total completion time, which is the completion time of the last job in the last machine
    return completion_times[num_jobs-1][num_machines-1]


## NEH

In [ ]:
def order_jobs_in_descending_order_of_total_completion_time(processing_times):
    total_completion_time = processing_times.sum(axis=1)
    return np.argsort(total_completion_time, axis=0).tolist()

def insertion(sequence, position, value):
    new_seq = sequence[:]
    new_seq.insert(position, value)
    return new_seq
def neh_algorithm(processing_times):
    ordered_sequence = order_jobs_in_descending_order_of_total_completion_time(processing_times)
    # Define the initial order
    J1, J2 = ordered_sequence[:2]
    sequence = [J1, J2] if evaluate_sequence([J1, J2], processing_times) < evaluate_sequence([J2, J1], processing_times) else [J2, J1]
    del ordered_sequence[:2]
    # Add remaining jobs
    for job in ordered_sequence:
        Cmax = float('inf')
        best_sequence = []
        for i in range(len(sequence)+1):
            new_sequence = insertion(sequence, i, job)
            Cmax_eval = evaluate_sequence(new_sequence, processing_times)
            if Cmax_eval < Cmax:
                Cmax = Cmax_eval
                best_sequence = new_sequence
        sequence = best_sequence
    return sequence, Cmax

## Gupta

In [ ]:
def sign(x):
    if x > 0:
        return 1
    elif x < 0:
        return -1
    else:
        return 0

def min_gupta(job, processing_times):
    m = np.inf
    _, machines = processing_times.shape
    for i in range(machines-1):
        k = processing_times[job][i] + processing_times[job][i+1]
        if (k < m):
            m = k
    return m

def gupta_heuristic(processing_times):
    jobs, machines = processing_times.shape
    f = []
    total_times = []
    for i in range(jobs):
        fi = sign(processing_times[i][0] - processing_times[i][machines-1]) / min_gupta(i,processing_times)
        f.append(fi)
        total_time = sum(processing_times[i])
        total_times.append(total_time)
    order = sorted(range(jobs), key=lambda k: (f[k], total_times[k]))
    cmax =  evaluate_sequence(order, processing_times)
    return order,cmax

# CDS

In [ ]:

def johnson_method(processing_times):
    # Nombre de tâches et de machines
    tâches, machines = processing_times.shape
    # Copie des temps de traitement
    temps_de_traitement_copie = processing_times.copy()
    # Valeur maximale pour la comparaison
    maximum = processing_times.max() + 1
    # Listes pour stocker les indices des tâches pour chaque machine
    machine_1 = []
    machine_2 = []

    # Vérifie que le nombre de machines est égal à 2, sinon lève une exception
    if machines != 2:
        raise Exception("La méthode de Johnson fonctionne uniquement avec deux machines")

    # Itère sur le nombre de tâches
    for i in range(tâches):
        # Trouve le temps de traitement minimum restant
        minimum = temps_de_traitement_copie.min()
        # Trouve la position du temps de traitement minimum
        position = np.where(temps_de_traitement_copie == minimum)

        # Si le minimum se trouve sur la première machine, ajoute l'indice de la tâche à machine_1
        if position[1][0] == 0:
            machine_1.append(position[0][0])
        # Sinon, ajoute l'indice de la tâche à machine_2
        else:
            machine_2.insert(0, position[0][0])

        # Marque le temps de traitement minimum comme traité en lui attribuant une valeur maximale
        temps_de_traitement_copie[position[0][0]] = maximum

    # Concatène les deux listes des indices des tâches et retourne la séquence optimisée
    return machine_1 + machine_2


def CDS_heuristic(processing_times):
    # Nombre de tâches et de machines
    tâches, machines = processing_times.shape
    # Nombre de paires de machines
    nombre_paires = machines - 1
    # Matrice pour stocker les temps de traitement pour la méthode de Johnson
    temps_de_traitement_johnson = np.zeros((tâches, 2))
    # Meilleur coût initialisé à une valeur infinie
    meilleur_coût = np.inf
    # Meilleure séquence initialisée à une liste vide
    meilleure_séquence = []

    # Itère sur chaque paire de machines
    for k in range(nombre_paires):
        # Calcule les temps de traitement cumulatifs pour la méthode de Johnson
        temps_de_traitement_johnson[:, 0] += processing_times[:, k]
        temps_de_traitement_johnson[:, 1] += processing_times[:, -k - 1]
        # Applique la méthode de Johnson pour obtenir une séquence optimisée
        séquence = johnson_method(temps_de_traitement_johnson)
        # Évalue le coût de la séquence obtenue
        coût = evaluate_sequence(séquence, processing_times)
        # Met à jour la meilleure séquence et le meilleur coût si le coût actuel est meilleur
        if coût < meilleur_coût:
            meilleur_coût = coût
            meilleure_séquence = séquence

    # Retourne la meilleure séquence et le meilleur coût obtenus par l'heuristique CDS
    return meilleure_séquence, meilleur_coût


# Bottelneck

In [ ]:
import numpy as np

def bottleneck_heuristic_lpt(processing_times):
    """
    This function implements the LPT-based bottleneck heuristic for flowshop scheduling using a NumPy array for processing times.

    Args:
        processing_times: A NumPy array of shape (num_jobs, num_machines) representing processing times.

    Returns:
        A list representing the job sequence.
    """
    # Number of jobs and machines
    num_jobs, num_machines = processing_times.shape

    # Calculate total processing time per machine
    machine_tpts = np.sum(processing_times, axis=0)  # Sum along job axis
    print(machine_tpts)
    # Identify bottleneck machine
    bottleneck_machine = np.argmax(machine_tpts)
    print(bottleneck_machine)
    # Sort jobs by processing time on bottleneck machine (descending order)
    job_lpt = np.argsort(processing_times[:, bottleneck_machine])[::-1]  # Get job indices sorted by processing time on bottleneck machine (descending)
    print(job_lpt)
    # Schedule jobs using insertion heuristic
    schedule = []
    completion_times = np.zeros(num_machines)  # Keeps track of completion times on each machine
    for job_index in job_lpt:
        insertion_position = 0
        for machine in range(num_machines):
            if completion_times[machine] <= completion_times[insertion_position]:
                insertion_position = machine
        schedule.append(job_index)  # Append job index to schedule
        for machine in range(num_machines):
            completion_times[machine] = max(completion_times[machine], completion_times[insertion_position]) + processing_times[job_index, machine]

    makespan = max(completion_times)  # Maximum completion time among all machines
    return schedule, makespan

# Example usage with your NumPy array
processing_times = np.array([[54, 83, 15, 71, 77, 36, 53, 38, 27, 87, 76, 91, 14, 29, 12, 77, 32, 87, 68, 94],
                     [79,  3, 11, 99, 56, 70, 99, 60,  5, 56,  3, 61, 73, 75, 47, 14, 21, 86,  5, 77],
                     [16, 89, 49, 15, 89, 45, 60, 23, 57, 64,  7,  1, 63, 41, 63, 47, 26, 75, 77, 40],
                     [66, 58, 31, 68, 78, 91, 13, 59, 49, 85, 85,  9, 39, 41, 56, 40, 54, 77, 51, 31],
                     [58, 56, 20, 85, 53, 35, 53, 41, 69, 13, 86, 72,  8, 49, 47, 87, 58, 18, 68, 28]])


job_sequence, makespan = bottleneck_heuristic_lpt(np.transpose(processing_times))
print("Job Sequence:", job_sequence)
print("Makespan:", makespan)



[1121 1000  947 1081 1004]
0
[19 11  9 17  1  4 15 10  3 18  0  6  7  5 16 13  8  2 12 14]
Job Sequence: [19, 11, 9, 17, 1, 4, 15, 10, 3, 18, 0, 6, 7, 5, 16, 13, 8, 2, 12, 14]
Makespan: 1179.0


In [ ]:
import numpy as np

# Let's assume we have 20 jobs and 5 machines
# The processing times are given in a 20x5 matrix (for simplicity, using random integers)

np.random.seed(0) # For reproducibility
processing_times =  np.array([[54, 83, 15, 71, 77, 36, 53, 38, 27, 87, 76, 91, 14, 29, 12, 77, 32, 87, 68, 94],
                     [79,  3, 11, 99, 56, 70, 99, 60,  5, 56,  3, 61, 73, 75, 47, 14, 21, 86,  5, 77],
                     [16, 89, 49, 15, 89, 45, 60, 23, 57, 64,  7,  1, 63, 41, 63, 47, 26, 75, 77, 40],
                     [66, 58, 31, 68, 78, 91, 13, 59, 49, 85, 85,  9, 39, 41, 56, 40, 54, 77, 51, 31],
                     [58, 56, 20, 85, 53, 35, 53, 41, 69, 13, 86, 72,  8, 49, 47, 87, 58, 18, 68, 28]])
processing_times=np.transpose(processing_times)
# Step 1: Identify the bottleneck machine
bottleneck_machine_index = np.argmax(np.sum(processing_times, axis=0))

# Step 2: Apply the LPT rule based on the bottleneck machine's processing times
job_indices = np.argsort(-processing_times[:, bottleneck_machine_index])

# The job_indices now represent the job sequence using the LPT rule

# Step 3: Since it's a permutable flowshop, the sequence is the same for all machines
# Displaying the sequence of jobs
print("Optimal sequence of jobs:", job_indices + 1)

# Step 4: We can now calculate the makespan using this sequence
# Initialize start and end times for each job on each machine
start_times = np.zeros(processing_times.shape)
end_times = np.zeros(processing_times.shape)

for i, job in enumerate(job_indices):
    for machine in range(processing_times.shape[1]):
        if machine == 0:
            # First machine, so start time is the end time of the previous job
            start_times[job][machine] = end_times[job_indices[i-1]][machine] if i > 0 else 0
        else:
            # Otherwise, it's the maximum of when the previous job finished on this machine
            # and when this job finished on the previous machine
            start_times[job][machine] = max(end_times[job][machine-1], end_times[job_indices[i]][machine])

        # Calculate end time for this job on this machine
        end_times[job][machine] = start_times[job][machine] + processing_times[job][machine]

# The makespan is the last end_time on the last machine
makespan = np.max(end_times)
print("Makespan:", makespan)

Optimal sequence of jobs: [20 12 18 10  2  5 16 11  4 19  1  7  8  6 17 14  9  3 13 15]
Makespan: 1334.0


In [ ]:
import numpy as np

def bottleneck(processing_times):
    """
    This function applies the bottleneck heuristic for flowshop scheduling.

    Args:
        processing_times: A NumPy array of shape (num_jobs, num_machines) representing processing times.

    Returns:
        job_sequence: A list representing the optimal job sequence.
        makespan: The makespan of the optimal schedule.
    """
    # Step 1: Identify the bottleneck machine
    bottleneck_machine_index = np.argmax(np.sum(processing_times, axis=0))

    # Step 2: Apply the LPT rule based on the bottleneck machine's processing times
    job_indices = np.argsort(-processing_times[:, bottleneck_machine_index])

    # Step 3: Calculate the makespan using the obtained sequence
    # Initialize start and end times for each job on each machine
    start_times = np.zeros(processing_times.shape)
    end_times = np.zeros(processing_times.shape)

    for i, job in enumerate(job_indices):
        for machine in range(processing_times.shape[1]):
            if machine == 0:
                # First machine, so start time is the end time of the previous job
                start_times[job][machine] = end_times[job_indices[i-1]][machine] if i > 0 else 0
            else:
                # Otherwise, it's the maximum of when the previous job finished on this machine
                # and when this job finished on the previous machine
                start_times[job][machine] = max(end_times[job][machine-1], end_times[job_indices[i]][machine])

            # Calculate end time for this job on this machine
            end_times[job][machine] = start_times[job][machine] + processing_times[job][machine]

    # The makespan is the last end_time on the last machine
    makespan = np.max(end_times)

    # The job sequence is obtained by adding 1 to the job indices (to start from 1)
    job_sequence = list(job_indices + 1)

    return job_sequence, makespan

# Example usage with the provided processing_times array
processing_times = np.array([[54, 83, 15, 71, 77, 36, 53, 38, 27, 87, 76, 91, 14, 29, 12, 77, 32, 87, 68, 94],
                              [79,  3, 11, 99, 56, 70, 99, 60,  5, 56,  3, 61, 73, 75, 47, 14, 21, 86,  5, 77],
                              [16, 89, 49, 15, 89, 45, 60, 23, 57, 64,  7,  1, 63, 41, 63, 47, 26, 75, 77, 40],
                              [66, 58, 31, 68, 78, 91, 13, 59, 49, 85, 85,  9, 39, 41, 56, 40, 54, 77, 51, 31],
                              [58, 56, 20, 85, 53, 35, 53, 41, 69, 13, 86, 72,  8, 49, 47, 87, 58, 18, 68, 28]])
processing_times = np.transpose(processing_times)

# Call the bottleneck function
job_sequence, makespan = bottleneck(processing_times)

# Output the results
print("Optimal sequence of jobs:", job_sequence)
print("Makespan:", makespan)


Optimal sequence of jobs: [20, 12, 18, 10, 2, 5, 16, 11, 4, 19, 1, 7, 8, 6, 17, 14, 9, 3, 13, 15]
Makespan: 1334.0


# tests

### NEH TEST

In [ ]:
import numpy as np
#-----------------------------------------------instance 2
matrix = np.array([
    [26, 38, 27, 88, 95, 55, 54, 63, 23, 45, 86, 43, 43, 40, 37, 54, 35, 59, 43, 50],
    [59, 62, 44, 10, 23, 64, 47, 68, 54, 9, 30, 31, 92, 7, 14, 95, 76, 82, 91, 37],
    [78, 90, 64, 49, 47, 20, 61, 93, 36, 47, 70, 54, 87, 13, 40, 34, 55, 13, 11, 5],
    [88, 54, 47, 83, 84, 9, 30, 11, 92, 63, 62, 75, 48, 23, 85, 23, 4, 31, 13, 98],
    [69, 30, 61, 35, 53, 98, 94, 33, 77, 31, 54, 71, 78, 9, 79, 51, 76, 56, 80, 72]
])


matrix = np.transpose(matrix)

neh_solution,makspan =neh_algorithm(matrix)
print(f'Results of NEH :')
print(f'Best sequence is {neh_solution} with a makespan of {makspan}.')


Results of NEH :
Best sequence is [5, 6, 14, 11, 12, 10, 2, 15, 19, 3, 16, 0, 8, 1, 9, 7, 18, 4, 13, 17] with a makespan of 1367.0.


### Gupta test



In [ ]:
import numpy as np

data_matrix = np.array([
    [26, 38, 27, 88, 95, 55, 54, 63, 23, 45, 86, 43, 43, 40, 37, 54, 35, 59, 43, 50],
    [59, 62, 44, 10, 23, 64, 47, 68, 54, 9, 30, 31, 92, 7, 14, 95, 76, 82, 91, 37],
    [78, 90, 64, 49, 47, 20, 61, 93, 36, 47, 70, 54, 87, 13, 40, 34, 55, 13, 11, 5],
    [88, 54, 47, 83, 84, 9, 30, 11, 92, 63, 62, 75, 48, 23, 85, 23, 4, 31, 13, 98],
    [69, 30, 61, 35, 53, 98, 94, 33, 77, 31, 54, 71, 78, 9, 79, 51, 76, 56, 80, 72]
])
matrix = np.array(data_matrix)
matrix = np.transpose(matrix)
ordre , makspan= gupta_heuristic(matrix)
print(f'Results of gupta :')
print(f'Best sequence is {ordre} with a makespan of {makspan}.')


Results of gupta :
Best sequence is [18, 5, 19, 14, 16, 2, 11, 8, 0, 6, 12, 10, 1, 4, 3, 15, 9, 17, 7, 13] with a makespan of 1380.0.


### CDS test


In [ ]:
import numpy as np
#-----------------------------------------------instance 2
matrix = np.array([
    [26, 38, 27, 88, 95, 55, 54, 63, 23, 45, 86, 43, 43, 40, 37, 54, 35, 59, 43, 50],
    [59, 62, 44, 10, 23, 64, 47, 68, 54, 9, 30, 31, 92, 7, 14, 95, 76, 82, 91, 37],
    [78, 90, 64, 49, 47, 20, 61, 93, 36, 47, 70, 54, 87, 13, 40, 34, 55, 13, 11, 5],
    [88, 54, 47, 83, 84, 9, 30, 11, 92, 63, 62, 75, 48, 23, 85, 23, 4, 31, 13, 98],
    [69, 30, 61, 35, 53, 98, 94, 33, 77, 31, 54, 71, 78, 9, 79, 51, 76, 56, 80, 72]
])


matrix = np.transpose(matrix)

neh_solution,makspan =CDS_heuristic(matrix)
print(f'Results of NEH :')
print(f'Best sequence is {neh_solution} with a makespan of {makspan}.')
